In [1]:
import pandas as pd
import numpy as np


In [2]:
# load data
data = pd.read_csv('./family_data.csv', index_col='family_id')
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


# preprocessing
1. compute Perference Cost matrix
2. compute Accounting Cost matrix
3. computer each family size
4. each family (choice_) DESIRED

In [3]:
# n代表家庭成员个数，如果满足choice需求，需要的panalty
def get_penalty(n ,choice):
    panalty = None
    if choice ==0:
        panalty = 0
    if choice ==1:
        panalty = 50
    if choice ==2:
        panalty = 50 +9 * n
    if choice ==3:
        panalty = 100 + 9 * n
    if choice ==4:
        panalty = 200 + 9 * n
    if choice ==5:
        panalty = 200 + 18 * n
    if choice ==6:
        panalty = 300 + 18 * n
    if choice ==7:
        panalty = 300 + 36 * n
    if choice ==8:
        panalty = 400 + 36 * n
    if choice ==9:
        panalty = 500 + (36 + 199) * n
    if choice > 9 :
        panalty = 500 + (36 + 398) * n
    return panalty



In [4]:
N_DAYS = 100
N_FAMILY = 5000
MIN_OCCUPANCY = 125
MAX_OCCUPANCY = 300

# compute pcost_mat, (0-99)的panalty
pcost_mat = np.full(shape=(N_FAMILY, 100), fill_value = 999999)
for i in range(N_FAMILY):
    f_num = data.loc[i, 'n_people']
    # 对于第i个家庭，初始化pcost = other choice下的panalty
    pcost_mat[i, :] = get_penalty(f_num, 10)
    for choice in range(10): 
        temp = data.loc[i][choice]
        panalty = get_penalty(f_num, choice)
        pcost_mat[i, temp-1] = panalty
        
pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [5]:
# compute Accounting Cost matrix
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1, MAX_OCCUPANCY+1), dtype=np.float64)
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        diff = abs(i - j)
        acost_mat[i, j] = max(0, (i - 125) / 400 * i ** (0.5 + diff/50.0))
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [6]:
FAMILY_SIZE = data['n_people'].values
DESIRED = data.values[:, :-1] - 1


# 使用LP和MIP求解，规划方案
1. 先使用LP 对绝大部分家庭进行规划
2. 再使用MIP 对剩余家庭进行规划
3. 会在两边结果 =》 最终规划


In [7]:
from ortools.linear_solver import pywraplp

In [8]:
# LP

def solveLP():
    solver = pywraplp.Solver('AssignmnetProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x = {}
    candidates = [[] for x in range(N_DAYS)]
    
    for i in range(N_FAMILY):
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            # define x[i, j]
            x[i, j ] = solver.BoolVar('x[%i, %i]' %(i, j))
    
    # 每天参观的人数
    daily_occupany = [solver.Sum([x[i, j] * FAMILY_SIZE[i] for i in candidates[j]]) \
                    for j in range(N_DAYS)]
    
    # 每个家庭，在10个choice中出现的情况
    family_presence = [solver.Sum(x[i, j] for j in DESIRED[i, :]) \
                        for i in range(N_FAMILY)]
    
    # 定义preference cost
    preference_cost = solver.Sum([pcost_mat[i, j] * x[i, j] for i in range(N_FAMILY)\
                                 for j in DESIRED[i, :]])
    solver.Minimize(preference_cost)
    
    # 任务增加的约束
    for j in range(N_DAYS - 1):
        solver.Add(daily_occupany[j] - daily_occupany[j + 1] <= 25)
        solver.Add(daily_occupany[j + 1] - daily_occupany[j] <= 25)    
    for i in range(N_FAMILY):
        solver.Add(family_presence[i] == 1)
    for j in range(N_DAYS):
        solver.Add(daily_occupany[j] >= MIN_OCCUPANCY)
        solver.Add(daily_occupany[j] <= MAX_OCCUPANCY)
    result = solver.Solve()
    
    temp = [(i, j, x[i, j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i, :] \
        if x[i, j].solution_value() > 0]
    df = pd.DataFrame(temp, columns=['family_id', 'day', 'result'])
    
    return df


In [9]:
%%time
result = solveLP()

CPU times: user 22 s, sys: 195 ms, total: 22.2 s
Wall time: 44.3 s


In [10]:
THRS = 0.999
assigned_df = result[result.result > THRS]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5075,4995,15,1.0
5076,4996,87,1.0
5077,4997,31,1.0
5078,4998,91,1.0


In [11]:
unassigned_df = result[(result.result > 1 - THRS) & (result.result < THRS)]
unassigned_df

,family_id,day,result
60,59,38,0.25
61,59,14,0.75
242,240,32,0.75
243,240,56,0.25
265,262,31,0.50
...,...,...,...
4990,4912,8,0.40
4992,4914,38,0.60
4993,4914,43,0.40
5040,4961,53,0.75


In [12]:
# 计算每天访问的人数
assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]
assigned_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5075,4995,15,1.0,4
5076,4996,87,1.0,2
5077,4997,31,1.0,6
5078,4998,91,1.0,5


In [13]:
# 按照day进行聚合
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([285, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 245, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       226, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       256, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 160, 136, 123, 125, 135, 158, 185, 158, 134, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 183, 158, 128, 125, 122, 124])

In [14]:
min_occupancy = np.array([max(0, MIN_OCCUPANCY - x) for x in occupancy])
min_occupancy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 1])

In [15]:
max_occupancy = np.array([MAX_OCCUPANCY - x for x in occupancy])
max_occupancy

array([ 15,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  55,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        74,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        44,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 140, 164, 177, 175, 165, 142, 115, 142, 166, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 117, 142, 172, 175, 178, 176])

In [16]:
def solveIP(families, min_occupancy, max_occupancy):
    solver = pywraplp.Solver('AssignmnetProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # 需要安排的家庭
    n_families = len(families)
    
    x = {}
    candidates = [[] for x in range(N_DAYS)]
    
    for i in families:
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            # define x[i, j]
            x[i, j ] = solver.BoolVar('x[%i, %i]' %(i, j))
    
    # 每天参观的人数
    daily_occupany = [solver.Sum([x[i, j] * FAMILY_SIZE[i] for i in candidates[j]]) \
                    for j in range(N_DAYS)]
    
    # 每个家庭，在10个choice中出现的情况
    family_presence = [solver.Sum(x[i, j] for j in DESIRED[i, :]) \
                        for i in families]
    
    # 定义preference cost
    preference_cost = solver.Sum([pcost_mat[i, j] * x[i, j] for i in families \
                                 for j in DESIRED[i, :]])
    solver.Minimize(preference_cost)
    
    # 任务增加的约束 
    for i in range(n_families):
        solver.Add(family_presence[i] == 1)
    for j in range(N_DAYS):
        solver.Add(daily_occupany[j] >= min_occupancy[j])
        solver.Add(daily_occupany[j] <= max_occupancy[j])
    result = solver.Solve()
    
    temp = [(i, j) for i in families for j in DESIRED[i, :] \
        if x[i, j].solution_value() > 0]
    
    df = pd.DataFrame(temp, columns=['family_id', 'day'])
    return df   
    

In [17]:
unassigned = unassigned_df.family_id.unique()
result = solveIP(unassigned, min_occupancy, max_occupancy)
result

,family_id,day
0,59,38
1,240,32
2,262,31
3,488,39
4,500,26
...,...,...
64,4850,4
65,4886,98
66,4912,17
67,4914,38


In [18]:
df = pd.concat((assigned_df[['family_id', 'day']], result)).sort_values('family_id')
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5075,4995,15
5076,4996,87
5077,4997,31
5078,4998,91


In [19]:
from numba import njit
# 计算perference cost
@njit(fastmath=True)
def pcost(prediction):
    # initialize
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = 0
    for (i, p) in enumerate(prediction):
        # 家庭人数
        n = FAMILY_SIZE[i]
        # 第i个家庭， p天cost
        penalty += pcost_mat[i, p]
        daily_occupancy[p] += n
    return penalty, daily_occupancy

# 计算accounting cost
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    num_out_of_range = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        
        n_pl = daily_occupancy[day + 1]
        n = daily_occupancy[day]
        num_out_of_range += (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        accounting_cost += acost_mat[n, n_pl]
    return accounting_cost, num_out_of_range

# 根据安排prediction
@njit(fastmath=True)
def cost_function(prediction):
    penalty, daily_occupancy = pcost(prediction)
    accounting_cost, num_out_of_range = acost(daily_occupancy)
    final_score = penalty + accounting_cost + num_out_of_range * 99999999
    return final_score
    
    

In [20]:
prediction = df.day.values
print(cost_function(prediction))

77480.2439961357


In [21]:
def save_result(pred, filename):
    result = pd.DataFrame(range(N_FAMILY), columns=['family_id'])
    result['assigned_day'] = pred + 1
    result.to_csv(filename, index=False)
    print(filename + ' saved')
    return result
result = save_result(prediction, 'submission1.csv')

submission1.csv saved


## 寻找更好的替代方案

In [22]:
def find_better(pred):
    fobs = np.argsort(FAMILY_SIZE)
    score = cost_function(pred)
    original_score = np.inf
    
    while score < original_score:
        original_score = score
        
        for family_id in fobs:
            for pick in range(10):
                day = DESIRED[family_id, pick]
                oldvalue = pred[family_id]
                pred[family_id] = day
                new_score = cost_function(pred)
                
                if new_score < score:
                    score = new_score
                else:
                    pred[family_id] = oldvalue
        print(score, end='\r')
    print(score)


In [23]:
new = prediction.copy()
find_better(new)

73751.58816071771


In [ ]:
result = save